In [ ]:
from scipy.stats import poisson
import pandas as pd
from datetime import datetime
from azure.storage.blob import *
import io

# These do not change

In [ ]:
# data1 = pd.read_csv('InnerEventTimes_Data_test.csv')
accountKey="EkMCD1bIrMp8g3+5DOpjFkRl3oDFXG83MmMSBrdVsP2iRa6s4hZSa3yis5l6grdYRPgOkFAUJHf6v+GardbUfw=="
accountName='iaprod'

# Below will needed to be changed

In [ ]:
containerName='poissoncraig'
blobName='data_b5a30808-d3e9-41f9-852e-ef39dcf31c7f_026a981b-e758-4809-9d2a-7b0050e0dc0a.txt'

# Data I/O

In [ ]:
block_blob_service=BlockBlobService(account_name=accountName,account_key=accountKey)
data1 = block_blob_service.get_blob_to_text(containerName,blobName,encoding='utf-8')

In [ ]:
data1 = data1.content

In [ ]:
df = pd.read_csv(io.StringIO(data1))

In [ ]:
data1 = df

In [ ]:
FinalData = pd.DataFrame([])
tup_now = datetime.now().timetuple()
dateToday =datetime.today()
today = datetime.strftime(dateToday,'%Y-%m-%d')
# data1 = pd.read_csv('./InnerEventTimes_Data_test.csv')
data1 = data1.sort_values(by=['CustomerAddressKey', 'Date2'], ascending=False)
AllButLastData1 = data1[data1.Date2 < today]
CustGroupABL = AllButLastData1.groupby(
    'CustomerAddressKey').mean().drop(['Row1','Daily_Revenue'],axis=1)
FinalData = data1.merge(
    CustGroupABL, on='CustomerAddressKey').rename(
    index=str, columns={'InnerEventTime_y':'AvgInnerEventTime','InnerEventTime_x':'DaySinceLastPurchase'}).drop('Row1',axis=1)
FinalData = FinalData.assign(p = 0).rename(index=str, columns={'p':'Probability'})
FinalData.Probability = (poisson.cdf(FinalData.DaySinceLastPurchase, FinalData.AvgInnerEventTime, loc=0))
writeFileName = 'innerEventTime_Current.csv'

## RETURN RESULTS TO BLOB
# This container and account must already be created, although with more code you could create them in python

blobName = writeFileName
output = io.StringIO()
output = FinalData.to_csv(index_label="idx", encoding = "utf-8", na_rep='NULL')

block_blob_service = BlockBlobService(account_name=accountName, account_key=accountKey) 
block_blob_service.create_blob_from_text(containerName, blobName, output)